In [9]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib as plt
import tensorflow as tf

In [10]:
# Import our input dataset
df = pd.read_csv('load_historical_price_by_day.csv')
df.head()

,date,open,high,low,close,ticker
0,36643,$143.00,$147.34,$143.00,$146.00,SPY
1,36644,$147.00,$147.86,$145.06,$145.09,SPY
2,36647,$146.56,$148.48,$145.84,$147.06,SPY
3,36648,$145.50,$147.13,$144.13,$144.13,SPY
4,36649,$144.00,$144.00,$139.78,$140.75,SPY


In [11]:
# Import our input dataset
df2 = pd.read_csv('load_recession_indicator_by_day.csv')
df2.head()

,date,recession_indicator,country
0,28491,t,cn
1,28492,t,cn
2,28493,t,cn
3,28494,t,cn
4,28495,t,cn


In [25]:
X = df2.copy()
X = X.drop("recession_indicator", axis=1)
X = pd.get_dummies(X, columns=["country"])
X.head()

,date,country_cn,country_de,country_eu,country_jp,country_uk,country_us
0,28491,1,0,0,0,0,0
1,28492,1,0,0,0,0,0
2,28493,1,0,0,0,0,0
3,28494,1,0,0,0,0,0
4,28495,1,0,0,0,0,0


In [26]:
y_cols = df2["recession_indicator"] 

y = pd.DataFrame()

y["recession_indicator"] = y_cols
y.head()

,recession_indicator
0,t
1,t
2,t
3,t
4,t


In [27]:
y['recession_indicator'].value_counts()

f    60090
t    46346
Name: recession_indicator, dtype: int64

In [28]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_cols, random_state=1)

In [30]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs", max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")



 Logistic regression model accuracy: 0.567
